In [1]:
from functools import reduce
import gc
import numpy as np

"""
교통류 변동지표_250901
STVM 계산(3차로 점선) + 지상부 추가 _ 분석시간 증가
    집계시간 : 5분
    분석시간 1800~8400
    검지기 : 280개
"""
import pandas as pd

import os


# FIX 값 모음
###################################################################################################################
start_interval = 1800
end_interval = 8400

weights = {
    "w1" : 1,
    "w2" : 1,
    "w3" : 1,
    "w4" : 1,
    "w5" : 1,
    "w6" : 1
}

vehicle_types = [100, 300, 630, 640, 650]
######## 검지기 #############
# 지상부_진입 검지기(결과 데이터는 지상부_진입의 6번째 데이터 부터 들어가므로 해당 검지기를 선정함)
enter_line = 6

# 본선부 검지기
main_line = 60

# 지상부_진출 검지기
exit_line = 275
############################


######## 램프 ###############
# 유입램프
input_ramp = 59

# 유출램프
output_ramp = 61
############################

######## 구간 ###############
# 진입부_지상
entry_point = [i for i in range(6,11)]

# 본선부
middle_point = [i for i in range(41,241)]

# 진출부_ 지상
exit_point = [i for i in range(271,276)]
############################

###################################################################################################################

# 함수 모음
###################################################################################################################

# 평균속도
def speed_mean(original_df):
    copy_df = original_df.copy()
    # TimeGroup, New_Measurement별 그룹화 및 속도 평균
    speed_mean_df = (
        copy_df.groupby(["TimeGroup", "New_Measurement"])
          .agg(V_mean=("v[km/h]", "mean"), V_count=("v[km/h]", "count"))
          .reset_index()
    )
    speed_mean_df["V_next"] = speed_mean_df.groupby("TimeGroup")["V_mean"].shift(-1)
    speed_mean_df["delta_V"] = (speed_mean_df["V_next"] - speed_mean_df["V_mean"]) / speed_mean_df["V_mean"]
    speed_mean_df["delta_V"] = speed_mean_df["delta_V"].fillna(0)
    return speed_mean_df

# 밀도
def density_mean(original_df):
    copy_df = original_df.copy()
    density_mean_df = copy_df.assign(K = copy_df["V_count"] * 12 / copy_df["V_mean"])
    density_mean_df["K_next"] = density_mean_df.groupby("TimeGroup")["K"].shift(-1)
    density_mean_df["delta_K"] = (density_mean_df["K_next"] - density_mean_df["K"]) / density_mean_df["K"]
    density_mean_df["delta_K"] = density_mean_df["delta_K"].fillna(0)
    return density_mean_df

# 가중 중차량 비율
def heavy_rate(original_df):
    copy_df = original_df.copy()
    # 진입부 검지기 선정
    measurement = enter_line

    heavy_df = (
        copy_df[copy_df["Vehicle type"].isin([630, 640, 650])]
        .groupby(["TimeGroup"])
        .size()
        .reset_index(name="heavy_count")
    )

    # TimeGroup별 총 차량 갯수 집계
    total_df = (
        copy_df.groupby(["TimeGroup"])
        .size()
        .reset_index(name="total_count")
    )

    heavy_rate_df = pd.merge(
        heavy_df,
        total_df,
        on=["TimeGroup"],
        how="left"
    )

    heavy_rate_df["rate"] = heavy_rate_df["heavy_count"] / heavy_rate_df["total_count"]
    return heavy_rate_df


# 진입 포화도
def entry_saturation(original_df):
    copy_df = original_df.copy()

    # 실측용량 C
    max_capacity = 6600
    entry_saturation_df = (
        copy_df.groupby(["TimeGroup", "New_Measurement"])
        .size()
        .reset_index(name="entry_volume")  # 차량 수를 entry_volume이라는 컬럼명으로
    )

    # 단위가 대/시 이기 때문에 현재 5분집계 * 12
    entry_saturation_df["Phi_진입"] = entry_saturation_df["entry_volume"] * 12 / max_capacity
    return entry_saturation_df


# 램프 유출입 비율
def rfr_rate(original_df):
    """
    1. 본선 1개의 검지기 data 수집(New_Measurement = 60)
    2. 위의 검지기 앞, 뒤 검지기 data (59 / 61)
    3. RFR 연산
    """
    main_df = (
        original_df[original_df["New_Measurement"] == main_line] # 60
        .groupby(["TimeGroup", "New_Measurement"])
        .size()
        .reset_index(name="main_line")
    )
    input_df = (
        original_df[original_df["New_Measurement"] == input_ramp] # 59
        .groupby(["TimeGroup", "New_Measurement"])
        .size()
        .reset_index(name="input_ramp")
    )
    output_df = (
        original_df[original_df["New_Measurement"] == output_ramp] # 61
        .groupby(["TimeGroup", "New_Measurement"])
        .size()
        .reset_index(name="output_ramp")
    )
    #main_df["RFR"] = (input_df["input_ramp"] + output_df["output_ramp"])/main_df["main_line"]
    # 유출램프 구현이 안 됐기 때문에 0값으로 fix
    main_df["RFR"] = 0
    return main_df

# 진출 원활율
def output_normality(original_df):
    copy_df = original_df.copy()
    # 6번 검지기(지상부_진입)에 들어온 차량의 번호, 시간
    entry_df = copy_df[copy_df["New_Measurement"] == enter_line][["VehNo", "t(Entry)"]]

    # 265번 검지기(지상부_진입)에 들어온 차량의 번호, 시간
    exit_df = copy_df[copy_df["New_Measurement"] == exit_line][["VehNo", "t(Entry)"]]

    # 차량 번호로 그룹화 후 시간의 최솟값(중복제거)
    entry_first = (
        entry_df.groupby("VehNo")["t(Entry)"].min()
        .reset_index()  # Series → DataFrame
        .rename(columns={"t(Entry)": "t_entry"})
    )
    exit_first = (
        exit_df.groupby("VehNo")["t(Entry)"].min()
        .reset_index()
        .rename(columns={"t(Entry)": "t_exit"})
    )

    # 지연시간
    merged = pd.merge(entry_first, exit_first, on="VehNo", how="inner")
    merged["delay_sec"] = merged["t_exit"] - merged["t_entry"]
    merged = merged[merged["delay_sec"] >= 0]  # 음수 제거

    # 지연시간(중앙값) → lag_bins
    if len(merged) and np.isfinite(np.nanmedian(merged["delay_sec"])): # delay_sec의 값이 유효하면
        lag_bins = int(round(np.nanmedian(merged["delay_sec"]) / 300)) # 단위시간으로 나눴을 때의 중간값 => 3(900초) => 진입한 차량이 진출을 통과하는데 평균 900초가 걸림
        print("lag_bins : ", lag_bins)
    else:
        lag_bins = 0  # 데이터 부족 시 동시간 매칭

    # TimeGroup별 진입/유출 카운트 집계
    entry_count = (original_df[original_df["New_Measurement"] == enter_line]
                .groupby("TimeGroup").size().reset_index(name="Q_in"))
    exit_count  = (original_df[original_df["New_Measurement"] == exit_line]
                .groupby("TimeGroup").size().reset_index(name="Q_out"))

    merged_counts = pd.merge(entry_count, exit_count, on="TimeGroup", how="left")


    # Q_out을 지연 시간만큼 shift
    merged_counts["Q_in_shift"] = merged_counts["Q_in"].shift(lag_bins).fillna(0)

    # 1−F(outrate) 계산
    merged_counts["F(outrate)"] = (merged_counts["Q_out"] / merged_counts["Q_in_shift"]).fillna(0)
    display("merged_counts : ", merged_counts)
    return merged_counts


def calculate_stvm(speed_df, density_df, heavy_df, entry_saturation_df, rfr_df, normality_df):


    # TimeGroup 기준으로  Merge
    merged_df = (
    speed_df[["TimeGroup", "New_Measurement", "delta_V"]]
    .merge(density_df[["TimeGroup", "New_Measurement", "delta_K"]], on=["TimeGroup", "New_Measurement"])
    .merge(heavy_df[["TimeGroup", "rate"]], on=["TimeGroup"])
    .merge(entry_saturation_df[["TimeGroup", "New_Measurement", "Phi_진입"]], on=["TimeGroup", "New_Measurement"])
    .merge(rfr_df[["TimeGroup", "RFR"]], on="TimeGroup")
    .merge(normality_df[["TimeGroup", "F(outrate)"]], on="TimeGroup")
    )

    merged_df["STVM"] = (
        weights["w1"] * merged_df["delta_V"] +
        weights["w2"] * merged_df["delta_K"] +
        weights["w3"] * merged_df["rate"] +
        weights["w4"] * merged_df["Phi_진입"] +
        weights["w5"] * merged_df["RFR"] +
        weights["w6"] * merged_df["F(outrate)"]
    )
    merged_df = modify_frame(merged_df)
    return merged_df


def calculate_z_score(stvm_df):
    copy_df = stvm_df.copy()

    # 평균
    mean_stvm = copy_df["STVM"].mean(skipna=True)

    # 표준편차
    std_stvm = copy_df["STVM"].std(skipna=True)

    # Z-Score 계산
    copy_df["Z-Score"] = (copy_df["STVM"] - mean_stvm) / std_stvm
    z_max = copy_df["Z-Score"].max()
    z_min = copy_df["Z-Score"].min()

    copy_df["환산점수"] = copy_df["Z-Score"].apply(lambda z : z_to_score(z, z_min, z_max))

    stvm_df = pd.pivot(copy_df, index="TimeGroup", columns= "New_Measurement", values="환산점수")
    return stvm_df

def modify_frame(df):
    # 지상부 1~5, 276~280 제거
    modify_df = df.copy()

    modify_df["StartTime"] = modify_df["TimeGroup"].str.split("~").str[0].astype(int)
    #modify_df = modify_df[(modify_df["StartTime"] >=3300) &(modify_df["StartTime"] < 6900)]
    modify_df = modify_df[(modify_df["StartTime"] >=3900) &(modify_df["StartTime"] < 7500)]

    remove_values = list(range(1, 6)) + list(range(276, 281))

    modify_df = modify_df[~modify_df["New_Measurement"].isin(remove_values)]
    modify_df["New_Measurement"] = modify_df["New_Measurement"].apply(
        lambda x: -5 + (x - 6) if 6 <= x <= 10 else (x - 10 if x >= 11 else x)
    )
    return modify_df


def variable_timegroup_avg(stvm_df):
    copy_df = stvm_df.copy()
    variable_time_df = copy_df.groupby("TimeGroup")[["delta_V", "delta_K", "rate", "Phi_진입", "RFR", "F(outrate)"]].mean()
    return variable_time_df

def variable_total_avg(variable_df):
    variable_total_df = pd.DataFrame([variable_df.mean(numeric_only=True)])
    return variable_total_df

def speed_density_avg(density_df):
    copy_df = density_df.copy()
    avg_df = modify_frame(copy_df)
    avg_df = pd.DataFrame([avg_df.mean(numeric_only=True)])
    avg_df = avg_df[["V_mean", "K"]]
    return avg_df

def pivot_table(df, value, preprocess=None):
    copy_df = df.copy()
    if preprocess :
        copy_df = preprocess(copy_df)
    return copy_df.pivot(index="TimeGroup", columns="New_Measurement", values=value)

def weighted_avg_speed(original_df):
    copy_df = original_df.copy()
    # TimeGroup, New_Measurement별 그룹화 및 속도 평균
    speed_mean_df = (
        copy_df.groupby(["TimeGroup", "New_Measurement", "Vehicle type"])
          .agg(V_mean=("v[km/h]", "mean"), V_count=("v[km/h]", "count"))
          .reset_index()
    )
    speed_mean_df["std_group"] = speed_mean_df.groupby(["TimeGroup", "New_Measurement"])["V_mean"].transform(lambda s: s.std(ddof=0))
    speed_mean_df["cv"] = speed_mean_df["std_group"] / speed_mean_df["V_mean"]
    speed_mean_df["w"] = 1 / speed_mean_df["cv"]
    speed_mean_df["w*v"] = speed_mean_df["w"] * speed_mean_df["V_mean"]

    weighted_result = (
        speed_mean_df.groupby(["TimeGroup","New_Measurement"])
          .apply(lambda g: g["w*v"].sum() / g["w"].sum())
          .reset_index(name="Weighted_Avg_Speed")
    )

    return weighted_result

def save_to_excel(excel_df, folder_path, file_name, i):
        excel_folder_path = os.path.join(folder_path, file_name)
        os.makedirs(excel_folder_path, exist_ok=True)
        excel_file_name = f"{file_name}_{i+1}.xlsx"
        excel_file_path = os.path.join(excel_folder_path, excel_file_name)
        excel_df.to_excel(excel_file_path, index=True)
        print(f"{excel_file_name} 생성 완료")

def z_to_score(z, z_min, z_max):
    if 1.645 <= z <= z_max:
        return 50 + ((95 + 5 * ((z - 1.645) / (z_max - 1.645))) * 0.5)
    elif 1.282 <= z < 1.645:
        return 50 + ((90 + 5 * ((z - 1.282) / (1.645 - 1.282))) * 0.5)
    elif 1.038 <= z < 1.282:
        return 50 + ((85 + 5 * ((z - 1.038) / (1.282 - 1.038))) * 0.5)
    elif 0.842 <= z < 1.038:
        return 50 + ((80 + 5 * ((z - 0.842) / (1.038 - 0.842))) * 0.5)
    elif 0.676 <= z < 0.842:
        return 50 + ((75 + 5 * ((z - 0.676) / (0.842 - 0.676))) * 0.5)
    elif 0.526 <= z < 0.676:
        return 50 + ((70 + 5 * ((z - 0.526) / (0.676 - 0.526))) * 0.5)
    elif 0.387 <= z < 0.526:
        return 50 + ((65 + 5 * ((z - 0.387) / (0.526 - 0.387))) * 0.5)
    elif 0.255 <= z < 0.387:
        return 50 + ((60 + 5 * ((z - 0.255) / (0.387 - 0.255))) * 0.5)
    elif -0.255 <= z < 0.255:
        return 50 + ((40 + 5 * ((z + 0.255) / (0.255 + 0.255))) * 0.5)
    elif -0.387 <= z < -0.255:
        return 50 + ((35 + 5 * ((z + 0.387) / (-0.255 + 0.387))) * 0.5)
    elif -0.526 <= z < -0.387:
        return 50 + ((30 + 5 * ((z + 0.526) / (-0.387 + 0.526))) * 0.5)
    elif -0.676 <= z < -0.526:
        return 50 + ((25 + 5 * ((z + 0.676) / (-0.676 + 0.842))) * 0.5)
    elif -0.842 <= z < -0.676:
        return 50 + ((20 + 5 * ((z + 0.842) / (-0.676 + 0.842))) * 0.5)
    elif -1.038 <= z < -0.842:
        return 50 + ((15 + 5 * ((z + 1.038) / (-0.842 + 1.038))) * 0.5)
    elif -1.282 <= z < -1.038:
        return 50 + ((10 + 5 * ((z + 1.282) / (-1.038 + 1.282))) * 0.5)
    elif -1.645 <= z < -1.282:
        return 50 + ((5 + 5 * ((z + 1.645) / (-1.282 + 1.645))) * 0.5)
    elif z_min <= z < -1.645:
        return 50 + ((0 + 5 * ((z + z_min) / (-1.645 + z_min))) * 0.5)
    else:
        return np.nan
###################################################################################################################

folder_path = r"C:\VISSIM_Workspace\테스트"
mer_list = [file for file in os.listdir(folder_path) if file.endswith(".mer")]

grouped_df = pd.DataFrame()
result_df = pd.DataFrame()


for i in range(len(mer_list)):
    mer_file = mer_list[i]
    with open(os.path.join(folder_path, mer_file), "r", encoding="utf-8", errors="ignore") as file:
            lines = file.readlines()
            # 데이터가 시작하는 인덱스 찾기
            data_start_idx = None

            for j, line in enumerate(lines):
                if "Measurem." in line:  # 컬럼명이 포함된 행 찾기
                    data_start_idx = j
                    break

            # 데이터프레임 생성
            if data_start_idx is not None:

                # 컬럼명 추출 및 공백 제거
                columns = [col.strip() for col in lines[data_start_idx].strip().split(";")]

                # 데이터 부분 추출 및 가공
                data_lines = lines[data_start_idx + 1:]  # 컬럼명 제외, 데이터 부분
                data = [line.strip().split(";") for line in data_lines if line.strip()]

                # 데이터프레임 생성
                df = pd.DataFrame(data, columns=columns)

                # 컬럼 내부 데이터 정수형 변환
                df = df.apply(pd.to_numeric, errors="coerce")

                original_df = df[(df["t(Entry)"] != -1.00)].reset_index(drop=True)


                #불필요 컬럼 제거
                original_df.drop(columns=["b[m/s2]", "tQueue", "Occ", "Pers"], inplace=True, errors="ignore")

                original_df["New_Measurement"] = original_df["Measurem."] % 1000

                bins = np.arange(start_interval, end_interval+1, 300)
                labels = [f"{start}~{start+300}" for start in bins[:-1]]  # 구간 라벨링

                # 구간 나누기 및 컬럼 추가
                original_df["TimeGroup"] = pd.cut(original_df["t(Entry)"], bins=bins, labels=labels, right=False)
                #display("original_df : ", original_df)

                # 속도변화율
                speed_df = speed_mean(original_df)

                # 밀도변화율
                density_df = density_mean(speed_df)

                # 중차량 혼입률
                heavy_df = heavy_rate(original_df)

                # 동적 포화도
                entry_saturation_df = entry_saturation(original_df)

                # 램프 간섭 영향률 = 0
                rfr_df = rfr_rate(original_df)

                # 진출 원활율
                normality_df = output_normality(original_df)

                # STVM 계산
                stvm_df = calculate_stvm(speed_df, density_df, heavy_df, entry_saturation_df, rfr_df, normality_df)
                display("stvm_df : ", stvm_df)

                # Z-Score 계산
                z_score_df = calculate_z_score(stvm_df)
                #display("z_score_df : ", z_score_df)
                #save_to_excel(z_score_df, folder_path, "환산점수", i)

                # STVM 피봇
                stvm_pivot_df = pivot_table(stvm_df, "STVM")
                #save_to_excel(stvm_pivot_df, folder_path, "STVM", i)

                # 속도값 피봇
                speed_pivot_df = pivot_table(speed_df, "V_mean", preprocess=modify_frame)
                #save_to_excel(speed_pivot_df, folder_path, "속도값", i)

                # 메모리 정리
                del df, original_df, speed_df, density_df, heavy_df, entry_saturation_df, rfr_df, normality_df, stvm_df, z_score_df
                gc.collect()


lag_bins :  3


'merged_counts : '

,TimeGroup,Q_in,Q_out,Q_in_shift,F(outrate)
0,1800~2100,493,470,0.0,inf
1,2100~2400,508,529,0.0,inf
2,2400~2700,476,472,0.0,inf
3,2700~3000,503,493,493.0,1.000000
4,3000~3300,450,468,508.0,0.921260
5,3300~3600,533,517,476.0,1.086134
6,3600~3900,518,459,503.0,0.912525
7,3900~4200,496,543,450.0,1.206667
8,4200~4500,501,465,533.0,0.872420
9,4500~4800,474,523,518.0,1.009653


'stvm_df : '

,TimeGroup,New_Measurement,delta_V,delta_K,rate,Phi_진입,RFR,F(outrate),STVM,StartTime
1965,3900~4200,-5,-0.001576,0.011675,0.046538,0.901818,0,1.206667,2.165122,3900
1966,3900~4200,-4,0.003459,0.002520,0.046538,0.910909,0,1.206667,2.170093,3900
1967,3900~4200,-3,-0.000133,-0.001851,0.046538,0.916364,0,1.206667,2.167584,3900
1968,3900~4200,-2,-0.006479,0.012525,0.046538,0.914545,0,1.206667,2.173795,3900
1969,3900~4200,-1,0.000942,0.008931,0.046538,0.920000,0,1.206667,2.183078,3900
...,...,...,...,...,...,...,...,...,...,...
5310,7200~7500,261,0.023322,-0.030655,0.043650,0.903636,0,1.016064,1.956018,7200
5311,7200~7500,262,0.015736,-0.011498,0.043650,0.896364,0,1.016064,1.960316,7200
5312,7200~7500,263,0.000183,0.013956,0.043650,0.900000,0,1.016064,1.973853,7200
5313,7200~7500,264,0.000515,0.007449,0.043650,0.912727,0,1.016064,1.980406,7200


...

lag_bins :  7


'merged_counts : '

,TimeGroup,Q_in,Q_out,Q_in_shift,F(outrate)
0,1800~2100,546,406,0.0,inf
1,2100~2400,505,436,0.0,inf
2,2400~2700,516,403,0.0,inf
3,2700~3000,511,425,0.0,inf
4,3000~3300,508,453,0.0,inf
5,3300~3600,551,424,0.0,inf
6,3600~3900,564,413,0.0,inf
7,3900~4200,520,445,546.0,0.815018
8,4200~4500,490,434,505.0,0.859406
9,4500~4800,555,467,516.0,0.905039


'stvm_df : '

,TimeGroup,New_Measurement,delta_V,delta_K,rate,Phi_진입,RFR,F(outrate),STVM,StartTime
1965,3900~4200,-5,0.010739,-0.018236,0.301350,0.945455,0,0.815018,2.054326,3900
1966,3900~4200,-4,0.008292,-0.008224,0.301350,0.938182,0,0.815018,2.054618,3900
1967,3900~4200,-3,-0.000372,0.004250,0.301350,0.938182,0,0.815018,2.058428,3900
1968,3900~4200,-2,0.000284,-0.004144,0.301350,0.941818,0,0.815018,2.054327,3900
1969,3900~4200,-1,0.007129,-0.024397,0.301350,0.938182,0,0.815018,2.037282,3900
...,...,...,...,...,...,...,...,...,...,...
5310,7200~7500,261,0.149572,-0.141947,0.307653,0.801818,0,0.809346,1.926443,7200
5311,7200~7500,262,0.086964,-0.084236,0.307653,0.790909,0,0.809346,1.910636,7200
5312,7200~7500,263,0.035744,-0.043430,0.307653,0.787273,0,0.809346,1.896586,7200
5313,7200~7500,264,0.023390,-0.013744,0.307653,0.780000,0,0.809346,1.906644,7200
